In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import os
from tqdm import tqdm
from skimage.feature import local_binary_pattern

# Set input/output folders
input_folder = "/content/drive/MyDrive/dp/data/sabbir"
output_folder = "/content/drive/MyDrive/dp/data/CLAHE-YCrCb-LBP"
os.makedirs(output_folder, exist_ok=True)

def preprocess_clahe_ycrcb_lbp(img_path, save_folder):
    # Read grayscale image
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        return

    filename = os.path.basename(img_path).split('.')[0]

    # ---- Step 1: CLAHE on Y channel (YCrCb) ----
    img_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    y_clahe = clahe.apply(y)

    ycrcb_clahe = cv2.merge((y_clahe, cr, cb))
    # Convert YCrCb to BGR, then BGR to GRAY
    img_bgr_clahe = cv2.cvtColor(ycrcb_clahe, cv2.COLOR_YCrCb2BGR)
    clahe_img = cv2.cvtColor(img_bgr_clahe, cv2.COLOR_BGR2GRAY)


    # ---- Step 2: Local Binary Pattern (LBP) ----
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(clahe_img, n_points, radius, method="uniform")

    # Normalize LBP to 0–255 for saving
    lbp_normalized = np.uint8(255 * (lbp - lbp.min()) / (lbp.max() - lbp.min()))


    cv2.imwrite(os.path.join(save_folder, f"{filename}_claheY_lbp.png"), lbp_normalized)



all_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png','.jpg','.jpeg'))]

for file in tqdm(all_files):
    preprocess_clahe_ycrcb_lbp(os.path.join(input_folder, file), output_folder)

print("✅ All images processed with CLAHE-YCrCb + LBP and saved in:", output_folder)